In [ ]:
from collections import OrderedDict

import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import LineString, Polygon
from skfem import Basis, ElementTriP0
from skfem.io import from_meshio
from tqdm import tqdm

from femwell.maxwell.waveguide import compute_modes
from femwell.mesh import mesh_from_OrderedDict
from femwell.thermal import solve_thermal


import shapely as shp


: 

In [ ]:
w_sim = 64 # * 2
h_clad = 4
h_box = 2
w_core = 1.0
h_core = 0.3
#h_heater = 0.14
h_bcb = 0.1
h_silicon = 280 # 0.5

d_pillar = 2.0 
w_pillar = 3.0

# w_heater = 2.0 + 2*(d_pillar + w_pillar)
# w_heater = 4.0
# offset_heater = -h_core / 2 + h_clad + 0.5

w_35_ridge = 4.0
w_35_mesa = 2 * (d_pillar + w_pillar)
h_35_ridge = 2.0
h_35_mesa = 0.5
h_air = 14.0

x0 = -w_sim / 2
y0 = -h_core / 2 - h_box - h_silicon
x1 = w_sim / 2
y1 = -h_core / 2 + h_clad + h_air
env = shp.box(x0, y0, x1, y1)


pillar_r = shp.box(+w_core/2 + d_pillar, -h_core / 2 - h_box, +w_core/2 + d_pillar + w_pillar, +h_core / 2)
pillar_l = shp.box(-w_core/2 - d_pillar - w_pillar, -h_core / 2 - h_box, -w_core/2 - d_pillar, +h_core / 2)
core = shp.box(-w_core/2, -h_core / 2, +w_core/2, +h_core/2)
bcb = shp.box(-w_sim/2, h_core/2, +w_sim/2, h_core/2 + h_bcb)
mesa = shp.box(-w_35_mesa/2, h_core/2 + h_bcb, +w_35_mesa/2, h_core/2 + h_bcb + h_35_mesa)
ridge = shp.box(-w_35_ridge/2, h_core/2 + h_bcb + h_35_mesa, +w_35_ridge/2, h_core/2 + h_bcb + h_35_mesa + h_35_ridge)
# heater = shp.box(-w_heater / 2, -h_heater / 2 + offset_heater, w_heater / 2, h_heater / 2 + offset_heater)
wafer = shp.box(-w_sim / 2, -h_core / 2 - h_box - h_silicon, w_sim / 2, -h_core / 2 - h_box)
air = shp.box(-w_sim /2, y1-h_air, +w_sim/2, y1)
polygons = OrderedDict(
    bottom=LineString(
        [
            (-w_sim / 2, -h_core / 2 - h_box - h_silicon),
            (w_sim / 2, -h_core / 2 - h_box - h_silicon),
        ]
    ),
    air=air,
    ridge=ridge,
    mesa=mesa,
    bcb = bcb,
    core = core,
    pillar_l=pillar_l,
    pillar_r=pillar_r,
    # heater = heater,
    wafer = wafer,
    oxide = shp.clip_by_rect(env, -np.inf, -np.inf, np.inf, np.inf)

)

resolutions=dict(
    core={"resolution": 0.04, "distance": 1},
    pillar_l={"resolution": 0.2, "distance": 1},
    pillar_r={"resolution": 0.2, "distance": 1},
    bcb={"resolution": 0.2, "distance": 1},
    mesa={"resolution": 0.1, "distance": 1},
    ridge={"resolution": 0.1, "distance": 1},
    air={"resolution": 0.4, "distance": 1},

    # heater={"resolution": 0.6, "distance": 1},
)


mesh = from_meshio(mesh_from_OrderedDict(polygons, resolutions, default_resolution_max=0.6))
mesh.draw().show()

In [ ]:
current = 60e-3
T_min = 25
T_max = 65

# currents = np.linspace(curr, curr, 1)
current_density = current / polygons["ridge"].area
neffs = []

tc_air = 0.025
tc_oxide = 1.38
tc_silicon = 148
tc_nitride = 43
#tc_heater = 28,
tc_mesa = 60
tc_ridge = 50
tc_bcb = 0.29

In [ ]:
fig, axs = plt.subplots(2,2,sharey=True, sharex=True)
# axs[0,0].set_aspect('equal')
# axs[0,1].set_aspect('equal')
# axs[1,0].set_aspect('equal')
# axs[1,1].set_aspect('equal')


In [ ]:
## No silicon pillar
tc_pillar = tc_oxide

basis0 = Basis(mesh, ElementTriP0(), intorder=4)
thermal_conductivity_p0 = basis0.zeros()
for domain, value in {
    "air": tc_air,
    "ridge": tc_ridge,
    "mesa": tc_mesa,
    "bcb": tc_bcb,
    "core": tc_nitride,
    "pillar_r": tc_pillar,
    "pillar_l": tc_pillar,
    "wafer": tc_silicon,
    "oxide": tc_oxide,
}.items():
    thermal_conductivity_p0[basis0.get_dofs(elements=domain)] = value
thermal_conductivity_p0 *= 1e-12  # 1e-12 -> conversion from 1/m^2 -> 1/um^2


In [ ]:

# basis, temperature = solve_thermal(
#     basis0,
#     thermal_conductivity_p0,
#     specific_conductivity={"heater": 2.3e6},
#     current_densities={"heater": current_density},
#     fixed_boundaries={"bottom": 25},
# )

basis, temperature = solve_thermal(
    basis0,
    thermal_conductivity_p0,
    specific_conductivity={"ridge": 2.3e6},
    current_densities={"ridge": current_density},
    fixed_boundaries={"bottom": 25},
)

tc_no_pillar = thermal_conductivity_p0*1e12


basis0_no_pillar = basis0
basis_no_pillar = basis
temp_no_pillar = temperature

In [ ]:
## With silicon pillar
tc_pillar = tc_silicon

basis0 = Basis(mesh, ElementTriP0(), intorder=4)
thermal_conductivity_p0 = basis0.zeros()
for domain, value in {
    "air": tc_air,
    "ridge": tc_ridge,
    "mesa": tc_mesa,
    "bcb": tc_bcb,
    "core": tc_nitride,
    "oxide": tc_oxide,
    "pillar_r": tc_pillar,
    "pillar_l": tc_pillar,
    "wafer": tc_silicon,
}.items():
    thermal_conductivity_p0[basis0.get_dofs(elements=domain)] = value
thermal_conductivity_p0 *= 1e-12  # 1e-12 -> conversion from 1/m^2 -> 1/um^2

tc_Pillar = thermal_conductivity_p0*1e12
basis, temperature = solve_thermal(
    basis0,
    thermal_conductivity_p0,
    specific_conductivity={"ridge": 2.3e6},
    current_densities={"ridge": current_density},
    fixed_boundaries={"bottom": 25},
)

basis0_pillar = basis0
basis_pillar = basis
temp_pillar = temperature


In [ ]:

basis0_no_pillar.plot(tc_no_pillar, shading="flat", ax = axs[0,0], colorbar=True)
basis_no_pillar.plot(temp_no_pillar, shading="gouraud", ax = axs[1,0], colorbar=True, vmin = T_min, vmax = T_max)
plt.show()

basis0_pillar.plot(tc_Pillar, shading="flat", ax = axs[0,1], colorbar=True)
basis_pillar.plot(temp_pillar, shading="gouraud", ax = axs[1,1], colorbar=True, vmin=T_min, vmax=T_max)
plt.show()


In [ ]:
axs[0,0].set_title("No pillar - Th. cond.")
axs[0,1].set_title("Si pillar - Th. cond.")

axs[1,0].set_title("No pillar - Temp. [C]")
axs[1,1].set_title("Si pillar - Temp. [C]")

axs[0,0].set_ylabel("µm")
axs[1,0].set_ylabel("µm")
axs[1,0].set_xlabel("µm")
axs[1,1].set_xlabel("µm")
fig

In [ ]:
axs[1,0].set_ylim([-10, 10])
axs[1,1].set_ylim([-10, 10])
fig

In [ ]:
regions_pillar = (
    air,
    ridge,
    mesa,
    bcb,
    core,
    pillar_l,
    pillar_r,
    wafer,
)

regions_no_pillar = (
    air,
    ridge,
    mesa,
    bcb,
    core,
    wafer,
)

cl = 'w'

regions = regions_pillar
for r in regions:
    #print(r.exterior)
    [x,y] = r.exterior.xy 
    axs[1, 1].plot(x, y,cl,linewidth=0.8)
    axs[0, 1].plot(x, y,cl,linewidth=0.8)

plt.show()
fig

regions = regions_no_pillar
for r in regions:
    #print(r.exterior)
    [x,y] = r.exterior.xy 
    axs[1, 0].plot(x, y,cl,linewidth=0.8)
    axs[0, 0].plot(x, y,cl,linewidth=0.8)

plt.show()
fig
